In [1]:
import jax
jax.config.update('jax_platforms', 'cpu')
import bmi
from minde.minde import MINDE
from minde.scripts.helper import get_data_loader, get_default_config

/home/foresti/miniconda3/envs/minde/lib/python3.12/site-packages/jaxtyping/__init__.py:244: UserWarning: jaxtyping version >=0.2.23 should be used with Equinox version >=0.11.1
  warnings.warn(


In [2]:
bmi.benchmark.BENCHMARK_TASKS

{'1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f9ed31167e0>,
 'normal_cdf-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f9ed31169c0>,
 '1v1-additive-0.1': <bmi.benchmark.task.Task at 0x7f9ed3116ea0>,
 '1v1-additive-0.75': <bmi.benchmark.task.Task at 0x7f9ed3116f00>,
 '1v1-bimodal-0.75': <bmi.benchmark.task.Task at 0x7f9ed3117bf0>,
 'wiggly-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f9ed3116f90>,
 'half_cube-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f9ed313c230>,
 'student-identity-1-1-1': <bmi.benchmark.task.Task at 0x7f9ed3116d50>,
 'asinh-student-identity-1-1-1': <bmi.benchmark.task.Task at 0x7f9ed313c590>,
 'swissroll_x-normal_cdf-1v1-normal-0.75': <bmi.benchmark.task.Task at 0x7f9ed313c8f0>,
 'multinormal-dense-2-2-0.5': <bmi.benchmark.task.Task at 0x7f9ed313cc50>,
 'multinormal-dense-3-3-0.5': <bmi.benchmark.task.Task at 0x7f9ed313d010>,
 'multinormal-dense-5-5-0.5': <bmi.benchmark.task.Task at 0x7f9ed313d070>,
 'multinormal-dense-25-25-0.5': <bmi.benchmark.tas

In [3]:
bmi.benchmark.BENCHMARK_TASKS
name_task = "1v1-normal-0.75"
name_task = "spiral-multinormal-sparse-25-25-2-2.0"
# name_task="spiral-normal_cdf-multinormal-sparse-5-5-2-2.0"
task = bmi.benchmark.BENCHMARK_TASKS[name_task]
task.mutual_information

1.0216512475319774

In [4]:
args = get_default_config()
args.type ="j"
args.importance_sampling = True
args.use_ema = True
args.checkpoint_dir = r"/home/foresti/minde/checkpoints"

In [5]:
task.dim_x
task.dim_y

25

In [6]:
X, Y = task.sample(10, seed=42)
var_list = {}

In [7]:
print(X.shape)
print(type(X))

(10, 25)
<class 'jaxlib.xla_extension.ArrayImpl'>


In [8]:
import numpy as np

n = 100


def generate_distribution(n, pmf, **kwargs):
    p = np.zeros((n,1))
    for i in range(n):
        p[i] = pmf(i, n, **kwargs)
    p = p/np.sum(p)
    return p

def poisson_pmf(i, n, **kwargs):
    l = kwargs['l']
    return l**i*np.exp(-l)/np.math.factorial(i)

def uniform_pmf(i, n, **kwargs):
    return 1/n

def random_pmf(i, n, **kwargs):
    return np.random.rand()

def binomial_pmf(i, n, **kwargs):
    p = kwargs['p']
    return np.math.comb(n,i)*p**i*(1-p)**(n-i)

p_0_a = generate_distribution(n, binomial_pmf, p=0.01)
p_0_b = generate_distribution(n, poisson_pmf, l=0.5)

# Calculate mutual information, not kl divergence

/tmp/ipykernel_1511304/436664852.py:25: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  return np.math.comb(n,i)*p**i*(1-p)**(n-i)
/tmp/ipykernel_1511304/436664852.py:15: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  return l**i*np.exp(-l)/np.math.factorial(i)


In [23]:
from mutinfo.distributions.base import UniformlyQuantized
from scipy.stats import norm  # Used as a base distribution (to be quantized), you can use any other having a `cdf` method.
from scipy.stats import bernoulli, poisson, multinomial
import numpy as np

dim = 25
mutual_information = 1.0/dim
mutual_information = 1.0
n_samples = 100000

# random_variable = UniformlyQuantized(mutual_information, norm(loc=0.0, scale=1.0))
# random_variable = UniformlyQuantized(mutual_information, bernoulli(0.5))
# random_variable = UniformlyQuantized(mutual_information, poisson(4.0))

X, Y = UniformlyQuantized(mutual_information, bernoulli(0.5)).rvs(n_samples)

X = X.reshape(-1, 1)
Y = Y.reshape(-1, 1)

print(X.shape)
print(Y.shape)

# x and y are one-dimensional; stack x with independent z to increase the dimensionality.

dist = np.load(r"/home/foresti/minde/discrete_distributions/best_distribution_dim10_mutinfo01.npy")

# Flatten the joint distribution
flat_dist = dist.flatten()

# Sample from the multinomial distribution
samples = multinomial.rvs(n=1, p=flat_dist, size=n_samples)

# Convert 1D indices back to 2D indices
indices = np.argmax(samples, axis=1)

X, Y = np.unravel_index(indices, dist.shape)

X = X.reshape(-1, 1)
Y = Y.reshape(-1, 1)

print(f"Some samples of X: {X[:10]}")
print(f"Some samples of Y: {Y[:10]}")

print(f"x_samples.shape: {X.shape}")
print(f"samples shape: {samples.shape}")

for i in range(15):
    tmp_x = bernoulli(0.5).rvs(n_samples)
    tmp_y = bernoulli(0.5).rvs(n_samples)
    X = np.hstack((X, tmp_x.reshape(-1, 1)))
    Y = np.hstack((Y, tmp_y.reshape(-1, 1)))

(100000, 1)
(100000, 1)
Some samples of X: [[5]
 [7]
 [6]
 [7]
 [6]
 [6]
 [8]
 [2]
 [3]
 [1]]
Some samples of Y: [[3]
 [8]
 [2]
 [5]
 [0]
 [2]
 [8]
 [1]
 [5]
 [9]]
x_samples.shape: (100000, 1)
samples shape: (100000, 100)


In [24]:
from minde.minde import MINDE
from minde.scripts.helper import get_data_loader, get_default_config

args = get_default_config()
args.type ="j"
args.importance_sampling = True
args.use_ema = True
args.checkpoint_dir = r"/home/foresti/minde/checkpoints"

var_list = {"x": X.shape[1], "y": Y.shape[1]}
model = MINDE(args, var_list=var_list)
val = model(X, Y, std=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type           | Params
---------------------------------------------
0 | score     | UnetMLP_simple | 42.8 K
1 | model_ema | EMA            | 42.8 K
---------------------------------------------
85.5 K    Trainable params
0         Non-trainable params
85.5 K    Total params
0.342     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/foresti/miniconda3/envs/minde/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/foresti/miniconda3/envs/minde/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Epoch:  1  GT:  Not given MINDE_estimate:  0.42 MINDE_sigma_estimate:  3.179
Epoch:  2  GT:  Not given MINDE_estimate:  0.402 MINDE_sigma_estimate:  3.142
Epoch:  3  GT:  Not given MINDE_estimate:  0.341 MINDE_sigma_estimate:  2.76
Epoch:  4  GT:  Not given MINDE_estimate:  0.257 MINDE_sigma_estimate:  2.289
Epoch:  5  GT:  Not given MINDE_estimate:  0.209 MINDE_sigma_estimate:  1.849
Epoch:  6  GT:  Not given MINDE_estimate:  0.184 MINDE_sigma_estimate:  1.369
Epoch:  7  GT:  Not given MINDE_estimate:  0.178 MINDE_sigma_estimate:  1.016
Epoch:  8  GT:  Not given MINDE_estimate:  0.182 MINDE_sigma_estimate:  0.914
Epoch:  9  GT:  Not given MINDE_estimate:  0.194 MINDE_sigma_estimate:  0.775
Epoch:  10  GT:  Not given MINDE_estimate:  0.213 MINDE_sigma_estimate:  0.631
Epoch:  11  GT:  Not given MINDE_estimate:  0.23 MINDE_sigma_estimate:  0.537
Epoch:  12  GT:  Not given MINDE_estimate:  0.25 MINDE_sigma_estimate:  0.579
Epoch:  13  GT:  Not given MINDE_estimate:  0.276 MINDE_sigma_est

...

Epoch:  199  GT:  Not given MINDE_estimate:  1.165 MINDE_sigma_estimate:  1.167
Epoch:  200  GT:  Not given MINDE_estimate:  1.173 MINDE_sigma_estimate:  1.212
Epoch:  201  GT:  Not given MINDE_estimate:  1.144 MINDE_sigma_estimate:  1.009
Epoch:  202  GT:  Not given MINDE_estimate:  1.162 MINDE_sigma_estimate:  1.092
Epoch:  203  GT:  Not given MINDE_estimate:  1.152 MINDE_sigma_estimate:  1.172
Epoch:  204  GT:  Not given MINDE_estimate:  1.188 MINDE_sigma_estimate:  1.109
Epoch:  205  GT:  Not given MINDE_estimate:  1.18 MINDE_sigma_estimate:  1.219
Epoch:  206  GT:  Not given MINDE_estimate:  1.21 MINDE_sigma_estimate:  1.244
Epoch:  207  GT:  Not given MINDE_estimate:  1.174 MINDE_sigma_estimate:  1.167
Epoch:  208  GT:  Not given MINDE_estimate:  1.184 MINDE_sigma_estimate:  1.136
Epoch:  209  GT:  Not given MINDE_estimate:  1.199 MINDE_sigma_estimate:  0.999
Epoch:  210  GT:  Not given MINDE_estimate:  1.206 MINDE_sigma_estimate:  1.097
Epoch:  211  GT:  Not given MINDE_estimate

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=250` reached.


Epoch:  249  GT:  Not given MINDE_estimate:  1.207 MINDE_sigma_estimate:  1.12
Estimated MI: 1.2069902062416076 ± 0.007543636600601131
Estimated MI (σ): 1.2096484184265137 ± 0.03475201919223078


UserWarning: stop here

In [28]:
import torch

results = []
dist = np.load(r"/home/foresti/minde/discrete_distributions/best_distribution_dim10_mutinfo01.npy")

random_states = np.random.randint(0, 10000, 10)

for i in range(10):

    # Flatten the joint distribution
    flat_dist = dist.flatten()

    # Sample from the multinomial distribution
    samples = multinomial.rvs(n=1, p=flat_dist, size=10000, random_state=random_states[i])

    # Convert 1D indices back to 2D indices
    indices = np.argmax(samples, axis=1)

    X, Y = np.unravel_index(indices, dist.shape)

    X = X.reshape(-1, 1)
    Y = Y.reshape(-1, 1)

    data = {"x": torch.tensor(X, dtype=torch.float16), "y": torch.tensor(Y, dtype=torch.float16)}

    results.append(model.compute_mi(data))


In [33]:
print(results)
res_mi = [r[0] for r in results]
res_mi_sigma = [r[1] for r in results]

print(f"mi: {np.mean(res_mi)}")
print(f"mi sigma: {np.mean(res_mi_sigma)}")

[(1.210412049293518, 1.1783383369445801), (1.2140458583831788, 1.3137072563171386), (1.1958403944969178, 1.113105869293213), (1.1936938524246217, 1.1951070785522462), (1.2039091825485229, 1.2243215560913085), (1.214068603515625, 1.203419017791748), (1.2202549815177917, 1.2400245666503906), (1.1992243885993958, 1.2543378829956056), (1.22505863904953, 1.2018019676208496), (1.193651258945465, 1.2153966903686524)]
mi: 1.2070159208774567
mi sigma: 1.2139560222625732


In [34]:
print(X.shape)

(10000, 1)
